In [28]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [29]:
# Drop unused columns
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [49]:
# OPTIMIZATION 1
# Categorize ASK_AMT column and bin rarities
bins = [(i*1000)**1.5 for i in range(20)]
application_df['ASK_AMT_BIN'] = pd.cut(application_df.ASK_AMT, bins).astype(str)
application_df.ASK_AMT_BIN = application_df.ASK_AMT_BIN.replace('nan', 'Other')
application_df.ASK_AMT_BIN.value_counts()

(0.0, 31622.777]              27480
(31622.777, 89442.719]         2068
(89442.719, 164316.767]        1068
Other                           939
(164316.767, 252982.213]        614
(252982.213, 353553.391]        454
(353553.391, 464758.002]        304
(464758.002, 585662.019]        244
(585662.019, 715541.753]        187
(715541.753, 853814.968]        156
(853814.968, 1000000.0]         147
(1000000.0, 1153689.733]        117
(1153689.733, 1314534.138]       90
(1482228.053, 1656502.339]       86
(1314534.138, 1482228.053]       75
(1656502.339, 1837117.307]       74
(2023857.703, 2216528.818]       59
(1837117.307, 2023857.703]       50
(2414953.416, 2618969.263]       45
(2216528.818, 2414953.416]       42
Name: ASK_AMT_BIN, dtype: int64

In [4]:
# Bin rarities
app_type_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = app_type_counts[app_type_counts <= 250].index
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

app_class_counts = application_df.CLASSIFICATION.value_counts()
replace_class = app_class_counts[app_class_counts <= 1500].index
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [5]:
# Encode categorical columns
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.to_list()

enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values))
application_df_encoded = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)

C:\Users\lself\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
